<a href="https://colab.research.google.com/github/Narotsit-Karki/ML/blob/main/film(frame_interpolation_for_large_motion).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import requests
import numpy as np
import mediapy as media

In [2]:
_MAX_F = float(np.iinfo(np.uint8).max)
def load_image(image_url):
  headers ={'User-agent':'Colab Sample (https://tensorflow.org) '}
  response = requests.get(image_url,headers=headers)
  image = response.content

  image_data = tf.io.decode_image(image,channels = 3)
  image_numpy = tf.cast(image_data,dtype = tf.float32).numpy()

  return image_numpy / _MAX_F


In [5]:
def pad_to_align(x,align):
  assert np.ndim(x) == 4,'dimension must be of size 4'
  assert align > 0, 'align must be a  +ve number'

  height , width = x.shape[-3:-1]

  height_to_pad = (align - height % align) if height % align != 0 else 0
  width_to_pad = (align - width % align ) if width % align != 0 else 0

  bbox_to_pad = {
      'offset_height': height_to_pad // 2,
      'offset_width':width_to_pad // 2,
      'target_width': width + width_to_pad,
      'target_height': height + height_to_pad
  }

  padded_x  = tf.image.pad_to_bounding_box(x,**bbox_to_pad)
  bbox_to_crop = {
      'offset_height': height_to_pad // 2,
      'offset_width':width_to_pad // 2,
      'target_width': width ,
      'target_height': height
  }

  return padded_x , bbox_to_crop

In [6]:
class Interpolation:

  def __init__(self,align = 64):
    self._align = align
    self._model = hub.load("https://tfhub.dev/google/film/1")

  def __call__(self,x0,x1,dt):
    if self._align is not None:
      x0 , bbox_to_crop = pad_to_align(x0,self._align)
      x1,_= pad_to_align(x1,self._align)

      inputs= {'x0':x0,'x1':x1,'time':dt}
      result = self._model(inputs,training = False)
      image = result['image']

      if self._align is not None:
        image = tf.image.crop_to_bounding_box(image,**bbox_to_crop)
      return image.numpy()



In [7]:
def recursive_generator(frame1,frame2,num_recursion,interpolator):
  if num_recursion == 0:
    yield frame1
  else:
    time = np.full(shape = (1,),fill_value = 0.5,dtype = np.float32)
    time = np.expand_dims(time,axis = 0)
    x0 = np.expand_dims(frame1,axis = 0)
    x1= np.expand_dims(frame2,axis=0)
    mid_frame = interpolator(x0,x1,time)[0]

    yield from recursive_generator(frame1,mid_frame,num_recursion-1,interpolator)
    yield from recursive_generator(mid_frame,frame2,num_recursion-1,interpolator)

def interpolate_recursively(frames):
  num_recursion=6
  interpolator = Interpolation()
  n = len(frames)
  for i in range(1,n):
    yield from recursive_generator(frames[i-1],frames[i],num_recursion,interpolator)

  yield frames[-1]





In [8]:
image_url_1 = "https://github.com/google-research/frame-interpolation/blob/main/photos/one.png?raw=true"
image_url_2 = "https://github.com/google-research/frame-interpolation/blob/main/photos/two.png?raw=true"
image_data_1 = load_image(image_url=image_url_1)
image_data_2 = load_image(image_url=image_url_2)
input_frames = [image_data_1,image_data_2]
interpolated_frames = list(interpolate_recursively(input_frames))

media.show_video(interpolated_frames,fps =30,title = "FILM Interpolation using TFhub")




FILM Interpolation using TFhub This browser does not support the video tag.
